In [1]:
import pandas as pd
import math

In [2]:
import sys
import os

In [3]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'models'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'scripts'))

In [4]:
from models.Person import Person, Sex

In [5]:
from scripts.utils import random_item

In [6]:
person: Person = Person(first_name="Wisner", 
                last_name="CELUCUS", 
                address="Delmas", 
                phone_1="33511661", 
                sex=Sex.male)

In [7]:
haitian_names = pd.read_csv("./inputs/haitian_names.csv", encoding="latin-1")
kobo_df = pd.read_csv("./inputs/kobo_local.csv", encoding="latin-1")

In [8]:
h_fnames = list(haitian_names['FirstName'])

In [9]:
h_mnames = list(haitian_names['MiddleName'])

In [10]:
h_lnames = list(haitian_names['LastName'])

In [11]:
def get_fullname():
    f_name = random_item(h_fnames)
    m_name = random_item(h_mnames)
    l_name = random_item(h_lnames).upper()
    name = ""
    if isinstance(m_name, float):
        name = (f_name + " " + l_name).strip()
    else:
        if m_name != 'nan':
            name = (f_name + " " + m_name + " " + l_name).strip()
        else:
            name = (f_name + " " + m_name + " " + l_name).strip()
    return name

In [12]:
print(get_fullname())

Dany PIERRE


In [15]:
#kobo_df

In [31]:
departments_with_communes = kobo_df.groupby(['Departements', 'Communes', ])['Section communale/Ville'].apply(lambda x: list(set(x))).reset_index()
# Convert to an array (list of dictionaries)
result = departments_with_communes.to_dict(orient='records')

# Print the result
#print(result)

In [32]:
communes_with_sections = []
for depart in result:
    communes_with_sections.append(
        {'name': depart["Communes"], "Department": depart['Departements'], "sections": depart['Section communale/Ville'] }
    )

In [33]:
len(communes_with_sections )

32

In [34]:
departments_with_communes = kobo_df.groupby(['Departements', 'Communes', 'Section communale/Ville'])['Communauté/Addresse'].apply(lambda x: list(set(x))).reset_index()
# Convert to an array (list of dictionaries)
result2 = departments_with_communes.to_dict(orient='records')

In [29]:
#kobo_df

In [37]:
communes_with_sections_locals = []
for depart in result2:
    communes_with_sections_locals.append(
        {'name': depart["Communes"], "Department": depart['Departements'], "sections": depart['Section communale/Ville'], 'communities':depart['Communauté/Addresse'] }
    )

In [39]:
#communes_with_sections_locals

In [51]:
departments_with_structure = (
    kobo_df.groupby(['Departements', 'Communes', 'Section communale/Ville', 'Communauté/Addresse'])['Coordonnée GPS']
    .first()  # Keep first occurrence of gps_location for each community
    .reset_index()
    .groupby(['Departements', 'Communes', 'Section communale/Ville'])  # Now group by Section
    .apply(lambda x: x[['Communauté/Addresse', 'Coordonnée GPS']].to_dict(orient='records'))  # Convert communities with GPS to dict
    .reset_index(name='Communauté/Addresse')
    .groupby(['Departements', 'Communes'])  # Now group by Commune
    .apply(lambda x: x[['Section communale/Ville', 'Communauté/Addresse']].to_dict(orient='records'))  # Convert sections to dict
    .reset_index(name='Section communale/Ville')
    .groupby('Departements')  # Finally, group by Department
    .apply(lambda x: x[['Communes', 'Section communale/Ville']].to_dict(orient='records'))  # Convert communes to dict
    .reset_index(name='Communes')
)

result3 = departments_with_structure.to_dict(orient='records')

# Print the result
#result3

C:\Users\User\AppData\Local\Temp\ipykernel_23168\714702907.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[['Communauté/Addresse', 'Coordonnée GPS']].to_dict(orient='records'))  # Convert communities with GPS to dict
C:\Users\User\AppData\Local\Temp\ipykernel_23168\714702907.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[['Section communale/Ville', 'Communauté/

In [49]:
#kobo_df